# GoNuts Juice Company

## Objective and Prerequisites



## Problem description

GoNuts manufactures different juices made entirely of various exotic nuts.Their primary market is China and they operate three plants located in Ethiopia,Tanzania, and Nigeria. You have been asked to help them determine where tomanufacture the two newest juices they offer, Gingko Nut and Kola Nut. Eachplant has a different variable cost structure and capacity for manufacturing the different juices. Also, each juice has an expected demand.

## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex business problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation for the facility location problem.

## Model Formulation

### Sets and Indices

$j \in Plants$: Index and set of plant location: Ethiopia, Tanzania, Nigeria.

$i \in Products$: Index and set of products: Ginko and Kola.

### Decision Variables

$0 \geq x_{i,j}$: This non-negative continuous variable determines the number of products made at plant $i \in Plants$ of product $j \in Products$.

$y_{j} \in \{0, 1 \}$: This variable is equal to 1 if we open the plant at location $j \in J$; and 0 otherwise.

### Objective Function

For the different variants of GoNuts we use two different objective functions

- **Variable costs**
\begin{equation}
Min\;z = \sum_i \sum_j c_{ij} x_{ij}
\end{equation}

Where $C_{ij}$ is the cost to produce product $j$ at location $i$ and $x_{ij}$ the number of units produced of product $i$ at location $j$

- **Variable costs + Fixed Plant Costs**

\begin{equation*}
Min\;z = \sum_i \sum_j c_{ij} x_{ij} + \sum_j f_j y_j
\end{equation*}

Where $C_{ij}$ is the cost to produce product $j$ at location $i$ and $x_{ij}$ the number of units produced of product $i$ at location $j$. $f_j$ is the fixed cost to open location $j$ and $y_j$ whether the plant is opened or not.

### Constraints

For the different variants we need at least the following constraints

- **Demand**. For each product $i \in Products$ ensure that its demand is fulfilled. That is, the sum of the products received of each product from all plants must be equal to minimal demand $D_i$:

\begin{equation*}
    \sum_{j} x_{ij}  >= D_i \;\; \forall i
\end{equation*}

- **Capacity**. We need to ensure that we can only ship from facility $j \in Plants$ up to the maximal capacity $c_j$

\begin{equation*}
    \sum_{i} x_{ij}  <= c_j \;\; \forall j
\end{equation*}


For some variants we need one or more of the following constraints:

- **Linking**
In this linking constraint multiplies the binary decision variable $y_j$ (0,1) with the capacity of the plant to link the binary variable with the continuous decision variable.
Any value of $X_{ij}$ can only be positive if $y_j$ is 1 as any other value would lead to a right hand side that is $> 0$

\begin{equation*}
    \sum_{i} x_{ij} - My_j  <= 0 \;\; \forall j
\end{equation*}

- **Mimimal production**

\begin{equation*}
    \sum_{i} x_{ij} >= L_j y_j   \;\; \forall j
\end{equation*}
Where $L_j$ is the minimum level of production at plant $j$.

- **Maximum number of plants**

\begin{equation*}
    \sum_{j} y_{j} <= N
\end{equation*}

The sum of decision variable $y_j$ is the number of plants in use.


In [1]:
!pip install gurobipy

You should consider upgrading via the 'C:\Users\artur\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

# tested with Gurobi v9.1.0 and Python 3.7.0

### Model Deployment

We now determine the MIP model for the gonuts problems, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to build facilities that minimizes total costs.

### GoNuts 1

In GoNuts 1 we simply look at the best production plant that fits capacity and meets demand.

In [3]:
PLANTS = ['Ethiopia', 'Tanzania','Nigeria']
PRODUCTS = ['Ginko','Kola']
cartesian_prod = list(product(PLANTS, PRODUCTS))
cartesian_prod

[('Ethiopia', 'Ginko'),
 ('Ethiopia', 'Kola'),
 ('Tanzania', 'Ginko'),
 ('Tanzania', 'Kola'),
 ('Nigeria', 'Ginko'),
 ('Nigeria', 'Kola')]

In [4]:
#Table with production costs as dictionary with tuple (Plant, Product) as key to the unit cost.
# You can access with UNIT_COST[('Ethiopia','Ginko')]

UNIT_COST = { ('Ethiopia', 'Ginko'): 21,
              ('Ethiopia', 'Kola'): 22.5,
              ('Tanzania', 'Ginko'): 22.5,
              ('Tanzania', 'Kola'): 24.5,
              ('Nigeria', 'Ginko'): 23,
              ('Nigeria', 'Kola'): 25.5}

In [5]:
#We store capacity as a simple dictionary with plant as key
CAPACITY = {'Ethiopia'  : 425,
            'Tanzania'  : 400,
            'Nigeria'   : 750}

In [6]:
#We store demand as a simple dictionary with product as key
DEMAND = {'Ginko' : 550,
          'Kola'  : 450}

In [7]:
#Define model
m = gp.Model('GoNuts1')

# Add variables for each plant and product combination.
# X will be a dictionary with (plant, product) as a key.
X = m.addVars(cartesian_prod, vtype=GRB.CONTINUOUS, name='PRODUCT_MADE')
X

Restricted license - for non-production use only - expires 2024-10-28


{('Ethiopia', 'Ginko'): <gurobi.Var *Awaiting Model Update*>,
 ('Ethiopia', 'Kola'): <gurobi.Var *Awaiting Model Update*>,
 ('Tanzania', 'Ginko'): <gurobi.Var *Awaiting Model Update*>,
 ('Tanzania', 'Kola'): <gurobi.Var *Awaiting Model Update*>,
 ('Nigeria', 'Ginko'): <gurobi.Var *Awaiting Model Update*>,
 ('Nigeria', 'Kola'): <gurobi.Var *Awaiting Model Update*>}

In [8]:
#Set objective function to to the product of number of units produced times unit costs.
# Multiply the number of units product made with production cost in dictionary
# Since X and UNIT_COST have the same keys, Gurobi can pull the values from UNIT_COST
m.setObjective(X.prod(UNIT_COST), GRB.MINIMIZE)

In [9]:
# Add constraint to cap the output from each plant
# at the maximum capacity of each plant.
# This generator below creates a constraint with the capacity for each plant
m.addConstrs((X.sum(plant)<=CAPACITY[plant] for plant in PLANTS),name='Capacity')

#And would be equivalent to the following for-loop
#for plant in PLANTS:
#  m.addConstr(X.sum(plant) <= CAPACITY[plant], name='Capacity_'+plant)

{'Ethiopia': <gurobi.Constr *Awaiting Model Update*>,
 'Tanzania': <gurobi.Constr *Awaiting Model Update*>,
 'Nigeria': <gurobi.Constr *Awaiting Model Update*>}

In [10]:
# Add constraint to make sure the demand for each product is met.
m.addConstrs((X.sum('*',product)>= DEMAND[product] for product in PRODUCTS), name='Demand')

{'Ginko': <gurobi.Constr *Awaiting Model Update*>,
 'Kola': <gurobi.Constr *Awaiting Model Update*>}

In [15]:
#You can write your model to a text file 'gonuts1.lp' that you can look at.
m.write('gonuts1.lp')

In [16]:
# Read model formulation
with open('gonuts1.lp') as f:
  print (f.read())

\ Model GoNuts1
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  21 PRODUCT_MADE[Ethiopia,Ginko] + 22.5 PRODUCT_MADE[Ethiopia,Kola]
   + 22.5 PRODUCT_MADE[Tanzania,Ginko] + 24.5 PRODUCT_MADE[Tanzania,Kola]
   + 23 PRODUCT_MADE[Nigeria,Ginko] + 25.5 PRODUCT_MADE[Nigeria,Kola]
Subject To
 Capacity[Ethiopia]: PRODUCT_MADE[Ethiopia,Ginko]
   + PRODUCT_MADE[Ethiopia,Kola] <= 425
 Capacity[Tanzania]: PRODUCT_MADE[Tanzania,Ginko]
   + PRODUCT_MADE[Tanzania,Kola] <= 400
 Capacity[Nigeria]: PRODUCT_MADE[Nigeria,Ginko]
   + PRODUCT_MADE[Nigeria,Kola] <= 750
 Demand[Ginko]: PRODUCT_MADE[Ethiopia,Ginko] + PRODUCT_MADE[Tanzania,Ginko]
   + PRODUCT_MADE[Nigeria,Ginko] >= 550
 Demand[Kola]: PRODUCT_MADE[Ethiopia,Kola] + PRODUCT_MADE[Tanzania,Kola]
   + PRODUCT_MADE[Nigeria,Kola] >= 450
Bounds
End



In [13]:
#Optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x89c3c709
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+02]
Presolve time: 0.01s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.000000e+03   0.000000e+00      0s
       5    2.2637500e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.263750000e+04


In [14]:
print(f"Optimal objective value: {m.objVal}")

print("\nProduction plan:")
for (plant, product), var in X.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Plant {plant} produces {var.x} of {product} .")

Optimal objective value: 22637.5

Production plan:
Plant Ethiopia produces 425.0 of Kola .
Plant Tanzania produces 375.0 of Ginko .
Plant Tanzania produces 25.0 of Kola .
Plant Nigeria produces 175.0 of Ginko .


### GoNuts 2

In GoNuts2 we add a fixed cost structure to incorporate the costs to open a plant.

In [18]:
#Store plant costs as cost with plant name as key
PLANT_COST = {'Ethiopia'  : 1500,
               'Tanzania'  : 2000,
               'Nigeria'   : 3000}

In [19]:
# MIP  model formulation
m = gp.Model('GoNuts2')

#Add variables
X =m.addVars(cartesian_prod, vtype=GRB.CONTINUOUS, name='PRODUCT_MADE')
# We add an additional binary decision variable
# that tracks wether an plant is open
y = m.addVars(PLANTS, vtype=GRB.BINARY, name='PLANT_OPEN')

In [20]:
# Add constraint to maximize the output from each plant
# at the maximum capacity of each plant.
# This generator below creates a constraint with the capacity for each plant


In [21]:
# Add constraint to make sure the demand for each product is met.


In [22]:
#Linking
#m.addConstrs((X.sum(plant) - y.prod(CAPACITY) <= 0 for plant in PLANTS), name='Linking')
# The following line of code would do the same effectively.
m.addConstrs((X.sum(plant) - y[plant]*CAPACITY[plant] <= 0 for plant in PLANTS), name='Linking')


{'Ethiopia': <gurobi.Constr *Awaiting Model Update*>,
 'Tanzania': <gurobi.Constr *Awaiting Model Update*>,
 'Nigeria': <gurobi.Constr *Awaiting Model Update*>}

In [23]:
#We now include the plant cost in the objective function.
m.setObjective(X.prod(UNIT_COST) + y.prod(PLANT_COST), GRB.MINIMIZE)

In [28]:
#Write your model to a text file 'gonuts2.lp' that you can look at.
m.write('gonuts2.lp')

In [29]:
#Read your model formulation
with open('gonuts2.lp') as f:
  print (f.read())

\ Model GoNuts2
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  21 PRODUCT_MADE[Ethiopia,Ginko] + 22.5 PRODUCT_MADE[Ethiopia,Kola]
   + 22.5 PRODUCT_MADE[Tanzania,Ginko] + 24.5 PRODUCT_MADE[Tanzania,Kola]
   + 23 PRODUCT_MADE[Nigeria,Ginko] + 25.5 PRODUCT_MADE[Nigeria,Kola]
   + 1500 PLANT_OPEN[Ethiopia] + 2000 PLANT_OPEN[Tanzania]
   + 3000 PLANT_OPEN[Nigeria]
Subject To
 Linking[Ethiopia]: PRODUCT_MADE[Ethiopia,Ginko]
   + PRODUCT_MADE[Ethiopia,Kola] - 425 PLANT_OPEN[Ethiopia] <= 0
 Linking[Tanzania]: PRODUCT_MADE[Tanzania,Ginko]
   + PRODUCT_MADE[Tanzania,Kola] - 400 PLANT_OPEN[Tanzania] <= 0
 Linking[Nigeria]: PRODUCT_MADE[Nigeria,Ginko] + PRODUCT_MADE[Nigeria,Kola]
   - 750 PLANT_OPEN[Nigeria] <= 0
Bounds
Binaries
 PLANT_OPEN[Ethiopia] PLANT_OPEN[Tanzania] PLANT_OPEN[Nigeria]
End



In [30]:
#Optimize model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 9 columns and 9 nonzeros
Model fingerprint: 0xa15e06e7
Variable types: 6 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]



Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [27]:
print(f"Optimal objective value: {m.objVal}")
for plant,var in y.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Use plant {plant}.")
print("\nProduction plan:")
for (plant, product), var in X.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Plant {plant} produces {var.x} of {product} .")

Optimal objective value: 0.0

Production plan:


### GoNuts 3

In GoNuts 3 we want to ensure that a miminal production takes place at each plant so we remain cost effective.

In [ ]:
#Store minimal production with plant name as key
MIN_PRODUCTION = {'Ethiopia'  : 100,
                  'Tanzania'  : 250,
                  'Nigeria'   : 600}

In [ ]:
# MIP  model formulation
m =

#Add variables
X =
# We add an additional binary decision variable
# that tracks wether an plant is open
y =

#We keep the same objective function as the previous 2 models.


# Add constraint to maximize the output from each plant
# at the maximum capacity of each plant.
# This generator below creates a constraint with the capacity for each plant


# Add constraint to make sure the demand for each product is met.


#Linking


In [ ]:
# Minimal production constraint per plant
# For each plant we require that the production
# of either types is above the minimums if we open that plant.
# If the plant is closed and y is 0, than production can be 0



In [ ]:
# Write model to file 'gonuts3.lp'


In [ ]:
# Read model formulation
with open('gonuts3.lp') as f:
  print (f.read())

In [ ]:
# Optimize model


In [ ]:
print(f"Optimal objective value: {m.objVal}")
for plant,var in y.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Use plant {plant}.")
print("\nProduction plan:")
for (plant, product), var in X.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Plant {plant} produces {var.x} of {product} .")

The Gurobi log provided indicates the successful optimization of a mathematical model using the Gurobi solver. Let's break down the key elements in the log:

- Gurobi Version Information:

  This section provides information about the Gurobi solver version, including the version number and build details. It also mentions the CPU model and thread count.
- Model Information:

  It describes the optimization model being solved:
  The model has 11 rows (constraints), 9 columns (variables), and 30 non-zero coefficients in its constraint matrix.
  The variable types consist of 9 integer variables (including 3 binary) and 0 continuous variables.
  Coefficient statistics provide information about the range of coefficient values in the model, objective function coefficients, variable bounds, and right-hand side (RHS) values for constraints.
  The presolve phase removed 3 rows and 1 column, which often simplifies the problem by eliminating redundant or unnecessary constraints or variables.
  Presolve time indicates the time taken for the presolve phase, which aims to simplify the problem without affecting its optimal solution.
- Initial Heuristic Solution:

  The solver found an initial heuristic solution with an objective value of 27925. This is often obtained using heuristic algorithms to provide an initial starting point for the optimization process.
- Root Relaxation:

   Before the solver enters the main branch-and-bound process, it performs a relaxation of the original problem called the "root relaxation." This relaxation allows integer variables to take non-integer values, essentially treating them as continuous variables.
  The result of the root relaxation is an objective value of approximately 27257.35 after 6 simplex iterations.
- Node Exploration:

  The solver proceeds to explore nodes in a branch-and-bound search tree.
  The "Nodes" column indicates the number of nodes explored.
  The "Current Node" column provides details about the current node being explored.
  The "Objective Bounds" column displays the current best-known upper and lower bounds on the objective function value.
  The "Gap" represents the optimality gap, which measures how close the current best solution is to the lower bound.
  The "It/Node" column indicates the average number of simplex iterations performed per node.
  In this case, the solver found an optimal solution with an objective value of 27425, meeting the specified optimality tolerance (1.00e-04).
- Thread Information:

  The solver utilized 2 threads out of the 2 available processors for parallel processing.
- Solution Summary:

  The "Solution count" indicates that the solver found two solutions during the optimization process.
  The "Optimal solution found" message confirms that an optimal solution was found within the specified tolerance.
  "Best objective" provides the final optimal objective function value, which is approximately 27425, and indicates that the optimality gap is 0.0000%, meaning that the solver proved that the solution is indeed optimal within the specified tolerance.

In summary, the Gurobi log demonstrates that the solver successfully found an optimal solution to the optimization problem, with an objective value of approximately 27425, meeting the specified optimality tolerance. The Simplex algorithm played a crucial role in this process, especially in the root relaxation and node exploration phases, where it iteratively improved the objective function value while respecting the problem's constraints.

(Interpretation by ChatGPT)


### GoNuts 4

Constrain the maximum number of plants

In [ ]:
MAX_NUMBER_OF_PLANTS = 2

In [ ]:
# MIP  model formulation
m =

#Add variables
X =
# We add an additional binary decision variable
# that tracks wether an plant is open
y =

#Unlike the previous 2 mondels, our objective function again only looks at
# the variable cost per unit.


# Add constraint to maximize the output from each plant
# at the maximum capacity of each plant.
# This generator below creates a constraint with the capacity for each plant


# Add constraint to make sure the demand for each product is met.


#Linking


In [ ]:
#We now add a constraint that take the sum of the binary (1,0) variables
# whether a plant is open or not and constraint that our
# set max number of plants


In [ ]:
# Write model to file 'gonuts4.lp'


In [ ]:
# Read model
with open('gonuts4.lp') as f:
  print (f.read())

In [ ]:
# Optimize model


In [ ]:
print(f"Optimal objective value: {m.objVal}")
for plant,var in y.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Use plant {plant}.")
print("\nProduction plan:")
for (plant, product), var in X.items():
    if (abs(var.x) > 0): #Only print if not 0
        print(f"Plant {plant} produces {var.x} of {product} .")

## Acknowledgements

GoNuts is an example created by Chris Caplice and and the MITx MicroMasters® Program in Supply Chain Management